In [1]:
import pandas as pd

plants_db = pd.read_csv("LOTUSDB_predictions_plants.csv")

In [2]:
from deepmol.pipeline import Pipeline
from deepmol.datasets import SmilesDataset
from tqdm import tqdm
import rdkit.rdBase as rkrb
import rdkit.RDLogger as rkl

import numpy as np

logger = rkl.logger()
logger.setLevel(rkl.ERROR)
rkrb.DisableLog('rdApp.error')

smiles = plants_db.SMILES
ids = plants_db.ID

predictions = []
last_i = 0

best_pipeline = Pipeline.load("sm_predictor_pipeline_motpe/trial_311")

bar = tqdm(total=len(smiles))
for i in range(10000, len(smiles), 10000):
    lotus_dataset = SmilesDataset(smiles=smiles[last_i:i], ids=ids[last_i:i])
    prediction = best_pipeline.predict(lotus_dataset)
    if i == 10000:
        predictions = prediction
    else:
        predictions = np.concatenate((predictions, prediction))

    last_i += 10000
    bar.update(10000)


lotus_dataset = SmilesDataset(smiles=smiles[last_i:], ids=ids[last_i:])
prediction = best_pipeline.predict(lotus_dataset)
predictions = np.concatenate((predictions, prediction))

2023-09-11 15:15:17.697673: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-11 15:15:18.492209: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Skipped loading modules with pytorch-lightning dependency, missing a dependency. No module named 'pytorch_lightning'
Skipped loading some Jax models, missing a dependency. jax requires jaxlib to be installed. See https://github.com/google/jax#installation for installation instructions.
100%|█████████▉| 190000/190273 [07:20<00:00, 428.56it/s]

In [4]:
labels = np.array(['C00073', 'C00078', 'C00079', 'C00082', 'C00235', 'C00341', 'C00353',
       'C00448', 'C01789', 'C03506', 'C00047', 'C00108', 'C00187', 'C00148',
       'C00041', 'C00129', 'C00062', 'C01852', 'C00049', 'C00135', 'C00223',
       'C00509', 'C00540', 'C01477', 'C05903', 'C05904', 'C05905', 'C05908',
       'C09762'])

labels_ = {
    'C00341': 'Geranyl diphosphate',
    'C01789': 'Campesterol',
    'C00078': 'Tryptophan',
    'C00049': 'L-Aspartate',
    'C00183': 'L-Valine',
    'C03506': 'Indoleglycerol phosphate',
    'C00187': 'Cholesterol',
    'C00079': 'L-Phenylalanine',
    'C00047': 'L-Lysine',
    'C01852': 'Secologanin',
    'C00407': 'L-Isoleucine',
    'C00129': 'Isopentenyl diphosphate',
    'C00235': 'Dimethylallyl diphosphate',
    'C00062': 'L-Arginine',
    'C00353': 'Geranylgeranyl diphosphate',
    'C00148': 'L-Proline',
    'C00073': 'L-Methionine',
    'C00108': 'Anthranilate',
    'C00123': 'L-Leucine',
    'C00135': 'L-Histidine',
    'C00448': 'Farnesyl diphosphate',
    'C00082': 'L-Tyrosine',
    'C00041': 'L-Alanine',
    'C00540': 'Cinnamoyl-CoA',
    'C01477': 'Apigenin',
    'C05903': 'Kaempferol',
    'C05904': 'Pelargonin',
    'C05905': 'Cyanidin',
    'C05908': 'Delphinidin',
    'C00389': 'Quercetin',
    'C01514': 'Luteolin',
    'C09762': "Liquiritigenin",
    'C00509': 'Naringenin',
    'C00223': 'p-Coumaroyl-CoA'
}

labels_names = np.array([labels_[label] for label in labels])
labels_names

array(['L-Methionine', 'Tryptophan', 'L-Phenylalanine', 'L-Tyrosine',
       'Dimethylallyl diphosphate', 'Geranyl diphosphate',
       'Geranylgeranyl diphosphate', 'Farnesyl diphosphate',
       'Campesterol', 'Indoleglycerol phosphate', 'L-Lysine',
       'Anthranilate', 'Cholesterol', 'L-Proline', 'L-Alanine',
       'Isopentenyl diphosphate', 'L-Arginine', 'Secologanin',
       'L-Aspartate', 'L-Histidine', 'p-Coumaroyl-CoA', 'Naringenin',
       'Cinnamoyl-CoA', 'Apigenin', 'Kaempferol', 'Pelargonin',
       'Cyanidin', 'Delphinidin', 'Liquiritigenin'], dtype='<U26')

In [5]:
ones = predictions == 1
labels_all = []
for i, prediction in enumerate(ones):
    labels_all.append(";".join(labels_names[prediction]))

In [6]:
plants_db["prediction version 2"] = labels_all

In [8]:
plants_db.to_csv("LOTUSDB_predictions_plants_version_2.csv", index=False)